# 상수도 관망 이상 감지를 위한 AI 모델 개발
- '2024 상수도 관망 이상 감지 AI 경진대회'는 데이터와 AI 기술을 활용하여 상수도 관망의 이상 징후와 누수를 탐지하는 것을 목표로 합니다. <br> 이 대회는 복잡한 상수도 관망 시스템에서 발생하는 다양한 데이터를 효과적으로 분석하고 실시간으로 이상을 감지할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다.
- 이 대회의 궁극적 목적은 참가자들의 데이터 기반 이상 감지 역량을 강화하고, AI 기술이 실제 상수도 관리 시스템과 의사결정 과정에 어떻게 기여할 수 있는지 탐구하는 것입니다. <br> 개발된 AI 모델은 상수관망 디지털트윈 및 Water-Net 시스템에 통합되어 더욱 효율적이고 정확한 상수도 관리를 가능하게 할 것입니다.

## Baseline
- 본 베이스라인은 참가자분들께서 가장 기초적인 방법론을 통해 전체 프로세스를 이해하고 경험하실 수 있도록 구성되었습니다. <br> 데이터 로드부터 모델 학습, 추론, 그리고 최종 제출까지 이어지는 end-to-end 파이프라인의 경험을 제공하기 위함입니다.
- LSTM과 오토인코더를 활용한 시계열 이상치 탐지 파이프라인을 구현하고 있으며, 시계열 데이터의 특성을 고려해 윈도우 기반으로 학습 데이터를 생성하고,<br> 정상 패턴을 학습하여 이를 기반으로 이상치를 탐지하는 과정을 포함하고 있습니다.
- 제공되는 베이스라인 방법론이 본 task를 해결할 수 있는 유일한 방안은 아닙니다! 베이스라인 외에도 다양하고 유의미한 방법론들이 존재할테니,<br> 참가자 여러분들이 자유롭게 탐색하고 발전시켜 나가시기를 바랍니다.
- 데이터의 관망 정보를 활용하여 GNN(Graph Neural Networks) 등을 적용한다면, 더 나은 성능을 기대할 수도 있습니다.<br> 💡 여러분의 독창적인 아이디어로 의미 있는 성과를 만들어보세요! 🚀

# Import Library

In [1]:
import pandas as pd
import numpy as np
from types import SimpleNamespace
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os
from typing import List, Dict, Union

# Data Load

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
DATADIR = "/content/gdrive/MyDrive/kwater"

In [4]:
df_A = pd.read_csv(os.path.join(DATADIR,"train/TRAIN_A.csv"))
df_B = pd.read_csv(os.path.join(DATADIR,"train/TRAIN_B.csv"))

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/kwater/train/TRAIN_A.csv'

# Hyperparameter Setting

In [ ]:

config = {
    "WINDOW_GIVEN"      : 10080,   # 1 week
    "BATCH_SIZE"        : 64,
    "HIDDEN_DIM_LSTM"   : 1024,
    "NUM_LAYERS"        : 1,
    "EPOCHS"            : 3,
    "LEARNING_RATE"     : 1e-3,
    "DEVICE"            : "cuda",
    "DROPOUT"           : 0.2
}

CFG = SimpleNamespace(**config)

# Define Dataset

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df: pd.DataFrame, stride: int = 1, inference: bool = False) -> None:
        """
        Args:
            df: 입력 데이터프레임
            stride: 윈도우 스트라이드
            inference: 추론 모드 여부
        """
        self.inference = inference
        self.column_names = df.filter(regex='^P\\d+$').columns.tolist()
        self.file_ids = df['file_id'].values if 'file_id' in df.columns else None

        if inference:
            self.values = df[self.column_names].values.astype(np.float32)
            self._prepare_inference_data()
        else:
            self._prepare_training_data(df, stride)

    def _normalize_columns(self, data: np.ndarray) -> np.ndarray:
        """벡터화된 열 정규화"""
        mins = data.min(axis=0, keepdims=True)
        maxs = data.max(axis=0, keepdims=True)

        # mins와 maxs가 같으면 전체를 0으로 반환
        is_constant = (maxs == mins)
        if np.any(is_constant):
            normalized_data = np.zeros_like(data)
            normalized_data[:, is_constant.squeeze()] = 0
            return normalized_data

        # 정규화 수행
        return (data - mins) / (maxs - mins)

    def _prepare_inference_data(self) -> None:
        """추론 데이터 준비 - 단일 시퀀스"""
        self.normalized_values = self._normalize_columns(self.values)

    def _prepare_training_data(self, df: pd.DataFrame, stride: int) -> None:
        """학습 데이터 준비 - 윈도우 단위"""
        self.values = df[self.column_names].values.astype(np.float32)

        # 시작 인덱스 계산 (stride 적용)
        potential_starts = np.arange(0, len(df) - CFG.WINDOW_GIVEN, stride)

        # 각 윈도우의 마지막 다음 지점(window_size + 1)이 사고가 없는(0) 경우만 필터링
        accident_labels = df['anomaly'].values
        valid_starts = [
            idx for idx in potential_starts
            if idx + CFG.WINDOW_GIVEN < len(df) and  # 범위 체크
            accident_labels[idx + CFG.WINDOW_GIVEN] == 0  # 윈도우 다음 지점 체크
        ]
        self.start_idx = np.array(valid_starts)

        # 유효한 윈도우들만 추출하여 정규화
        windows = np.array([
            self.values[i:i + CFG.WINDOW_GIVEN]
            for i in self.start_idx
        ])

        # (윈도우 수, 윈도우 크기, 특성 수)로 한번에 정규화
        self.input_data = np.stack([
            self._normalize_columns(window) for window in windows
        ])

    def __len__(self) -> int:
        if self.inference:
            return len(self.column_names)
        return len(self.start_idx) * len(self.column_names)

    def __getitem__(self, idx: int) -> Dict[str, Union[str, torch.Tensor]]:
        if self.inference:
            col_idx = idx
            col_name = self.column_names[col_idx]
            col_data = self.normalized_values[:, col_idx]
            file_id = self.file_ids[idx] if self.file_ids is not None else None
            return {
                "column_name": col_name,
                "input": torch.from_numpy(col_data).unsqueeze(-1),  # (time_steps, 1)
                "file_id": file_id
            }

        window_idx = idx // len(self.column_names)
        col_idx = idx % len(self.column_names)

        return {
            "column_name": self.column_names[col_idx],
            "input": torch.from_numpy(self.input_data[window_idx, :, col_idx]).unsqueeze(-1)
        }

# Create Dataset

In [ ]:
train_dataset_A = TimeSeriesDataset(df_A, stride=60)
train_dataset_B = TimeSeriesDataset(df_B, stride=60)
train_dataset_A_B = torch.utils.data.ConcatDataset([train_dataset_A, train_dataset_B])

train_loader = torch.utils.data.DataLoader(train_dataset_A_B,
                                            batch_size=CFG.BATCH_SIZE,
                                            shuffle=True)

# Define Model

In [ ]:
class LSTM_AE(nn.Module):
    def __init__(self):
        super(LSTM_AE, self).__init__()

        # LSTM feature extractor
        self.lstm_feature = nn.LSTM(
            input_size=1,
            hidden_size=CFG.HIDDEN_DIM_LSTM,
            num_layers=CFG.NUM_LAYERS,
            batch_first=True,
            dropout=CFG.DROPOUT if CFG.NUM_LAYERS > 1 else 0
        )

        # Encoder modules
        self.encoder = nn.Sequential(
            nn.Linear(CFG.HIDDEN_DIM_LSTM, CFG.HIDDEN_DIM_LSTM//4),
            nn.ReLU(),
            nn.Linear(CFG.HIDDEN_DIM_LSTM//4, CFG.HIDDEN_DIM_LSTM//8),
            nn.ReLU(),
        )

        # Decoder modules
        self.decoder = nn.Sequential(
            nn.Linear(CFG.HIDDEN_DIM_LSTM//8, CFG.HIDDEN_DIM_LSTM//4),
            nn.ReLU(),
            nn.Linear(CFG.HIDDEN_DIM_LSTM//4, CFG.HIDDEN_DIM_LSTM),
        )

    def forward(self, x):
        _, (hidden, _) = self.lstm_feature(x)
        last_hidden = hidden[-1]  # (batch, hidden_dim)

        # AE
        latent_z = self.encoder(last_hidden)
        reconstructed_hidden = self.decoder(latent_z)

        return last_hidden, reconstructed_hidden

# Train AE

In [ ]:
def train_AE(model, train_loader, optimizer, criterion, n_epochs, device):
    train_losses = []
    best_model = {
        "loss": float('inf'),
        "state": None,
        "epoch": 0
    }

    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0.0

        with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{n_epochs}", unit="batch") as t:
            for batch in t:
                inputs = batch["input"].to(device)
                original_hidden, reconstructed_hidden = model(inputs) # [ Batch_size, HIDDEN_DIM_LSTM ]

                loss = criterion(reconstructed_hidden, original_hidden)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                t.set_postfix(loss=loss.item())

        avg_epoch_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_epoch_loss)

        print(f"Epoch {epoch + 1}/{n_epochs}, Average Train Loss: {avg_epoch_loss:.8f}")

        if avg_epoch_loss < best_model["loss"]:
            best_model["state"] = model.state_dict()
            best_model["loss"] = avg_epoch_loss
            best_model["epoch"] = epoch + 1

    return train_losses, best_model

In [ ]:
MODEL = LSTM_AE().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(MODEL.parameters(), lr=CFG.LEARNING_RATE)

In [ ]:
train_losses, best_model = train_AE(MODEL,
                                    train_loader=train_loader,
                                    optimizer=optimizer,
                                    criterion=criterion,
                                    n_epochs=CFG.EPOCHS,
                                    device=CFG.DEVICE)

Epoch 1/3:   0%|          | 0/314 [00:28<?, ?batch/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.31 GiB. GPU 0 has a total capacity of 10.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 12.36 GiB is allocated by PyTorch, and 13.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
INFER_MODEL = LSTM_AE().cuda()
INFER_MODEL.load_state_dict(best_model["state"])

<All keys matched successfully>

# Define threshold

In [ ]:
def calculate_and_save_threshold(MODEL, train_loader, percentile=98):
    MODEL.eval()
    train_errors = []
    with torch.no_grad():
        for batch in tqdm(train_loader):
            inputs = batch["input"].to(CFG.DEVICE)
            original_hidden, reconstructed_hidden = MODEL(inputs)
            mse_errors = torch.mean((original_hidden - reconstructed_hidden) ** 2, dim=1).cpu().numpy()
            train_errors.extend(mse_errors)

    threshold = np.percentile(train_errors, percentile)

    print(f"Threshold calculated and saved: {threshold}")
    return threshold

THRESHOLD = calculate_and_save_threshold(INFER_MODEL, train_loader)

100%|██████████| 314/314 [02:20<00:00,  2.23it/s]

Threshold calculated and saved: 1.415979906660425e-08


# Inference & Detect Anomaly

In [ ]:
def inference_test_files(MODEL, batch, device='cuda'):
    MODEL.eval()
    with torch.no_grad():
        inputs = batch["input"].to(device)
        original_hidden, reconstructed_hidden = MODEL(inputs)
        reconstruction_loss = torch.mean((original_hidden - reconstructed_hidden) ** 2, dim=1).cpu().numpy()
    return reconstruction_loss

def detect_anomaly(MODEL, test_directory):
    test_files = [f for f in os.listdir(test_directory) if f.startswith("TEST") and f.endswith(".csv")]
    test_datasets = []
    all_test_data = []

    for filename in tqdm(test_files, desc='Processing test files'):
        test_file = os.path.join(test_directory, filename)
        df = pd.read_csv(test_file)
        df['file_id'] = filename.replace('.csv', '')
        individual_df = df[['timestamp', 'file_id'] + df.filter(like='P').columns.tolist()]
        individual_dataset = TimeSeriesDataset(individual_df, inference=True)
        test_datasets.append(individual_dataset)

        all_test_data.append(df)

    combined_dataset = torch.utils.data.ConcatDataset(test_datasets)

    test_loader = torch.utils.data.DataLoader(
        combined_dataset,
        batch_size=256,
        shuffle=False
    )

    reconstruction_errors = []
    for batch in tqdm(test_loader):
        reconstruction_loss = inference_test_files(MODEL, batch, CFG.DEVICE)

        for i in range(len(reconstruction_loss)):
            reconstruction_errors.append({
                "ID": batch["file_id"][i],
                "column_name": batch["column_name"][i],
                "reconstruction_error": reconstruction_loss[i]
            })

    errors_df = pd.DataFrame(reconstruction_errors)

    flag_columns = []
    for column in sorted(errors_df['column_name'].unique()):
        flag_column = f'{column}_flag'
        errors_df[flag_column] = (errors_df.loc[errors_df['column_name'] == column, 'reconstruction_error'] > THRESHOLD).astype(int)
        flag_columns.append(flag_column)

    errors_df_pivot = errors_df.pivot_table(index='ID',
                                          columns='column_name',
                                          values=flag_columns,
                                          aggfunc='first')
    errors_df_pivot.columns = [f'{col[1]}' for col in errors_df_pivot.columns]
    errors_df_flat = errors_df_pivot.reset_index()

    errors_df_flat['flag_list'] = errors_df_flat.loc[:, 'P1':'P' + str(len(flag_columns))].apply(lambda x: x.tolist(), axis=1).apply(lambda x: [int(i) for i in x])
    return errors_df_flat[["ID", "flag_list"]]

In [ ]:
C_list = detect_anomaly(INFER_MODEL, test_directory="/workspace/Storage/kwater_2024_4/Data/raw/test/C")
D_list = detect_anomaly(INFER_MODEL, test_directory="/workspace/Storage/kwater_2024_4/Data/raw/test/D")
C_D_list = pd.concat([C_list, D_list])

100%|██████████| 65/65 [00:50<00:00,  1.30it/s]


# Prepare Submission File

In [ ]:
sample_submission = pd.read_csv("./sample_submission.csv")
# 매핑된 값으로 업데이트하되, 매핑되지 않은 경우 기존 값 유지
flag_mapping = C_D_list.set_index("ID")["flag_list"]
sample_submission["flag_list"] = sample_submission["ID"].map(flag_mapping).fillna(sample_submission["flag_list"])

sample_submission.to_csv("./baseline_submission.csv", index=False)